In [9]:
import pandas as pd
import re

In [84]:
tramontano_data_file = '../data/tramontano2018/41564_2018_123_MOESM3_ESM.xlsx'
strain_table = pd.read_excel(tramontano_data_file, 
                             sheet_name = 'S1. Selected gut bacteria')
media_table = pd.read_excel(tramontano_data_file, 
                             sheet_name = 'S2. Media composition',
                             skiprows = 2)
media_conditions = media_table.columns
media_conditions = [cond for cond in media_conditions if cond not in ['Class','Compound']]
media_table = media_table.fillna(0) # replace NA with 0
annotated_growth_data = pd.read_excel(tramontano_data_file,
                                      sheet_name = 'S3. Annotated data',
                                      skiprows = 2)
# Load the growth matrix. When growth was observed (based on thresholding),
# mean and (stdev) are included. When growth was not observed, 0 was entered
growth_matrix = pd.read_excel(tramontano_data_file,
                                      sheet_name = 'S4. Growth matrix',
                                      index_col=0)
# Load sample size sheet. Really unclear what the format is here. Looks like
# # of biological replicates in first and second batch, but not sure
# what the number in parentheses means.
sample_sizes = pd.read_excel(tramontano_data_file,
                                      sheet_name = 'S5. Sample sizes',
                                      index_col=0)

# additional sheets from the data aren't needed

In [86]:
growth_matrix.index

Index(['A. muciniphila', 'A. odontolyticus', 'A. putredinis', 'A. shahii',
       'B. adolescentis', 'B. animalis subsp. lactis BI-07',
       'B. animalis subsp. lactis BL-04', 'B. caccae', 'B. clarus',
       'B. coprocola', 'B. crossotus', 'B. dorei', 'B. eggerthii',
       'B. fragilis', 'B. fragilis enterotoxigenic', 'B. fragilis HM-20',
       'B. fragilis HM-709', 'B. fragilis HM-710', 'B. fragilis HM-711',
       'B. fragilis HM-712', 'B. fragilis HM-713', 'B. fragilis HM-714',
       'B. hansenii', 'B. longum subsp. infantis', 'B. longum subsp. longum',
       'B. obeum', 'B. ovatus', 'B. stercoris', 'B. thetaiotaomicron',
       'B. uniformis', 'B. uniformis HM-715', 'B. uniformis HM-716',
       'B. vulgatus', 'B. vulgatus HM-720', 'B. wadsworthia',
       'B. xylanisolvens', 'C. aerofaciens', 'C. bolteae', 'C. catus',
       'C. comes', 'C. leptum', 'C. perfringens C36', 'C. perfringens S107',
       'C. ramosum', 'C. saccharolyticum', 'D. formicigenerans', 'D. piger',
    

In [87]:
study_organisms = ['B. longum subsp. infantis',
                  'B. longum subsp. longum',
                  'R. gnavus',
                  'C. saccharolyticum',
                  'L. plantarum',
                  'L. lactis',
                  'B. vulgatus HM-720',
                  'B. clarus',
                  'P. merdae',
                  'P. distasonis']

In [62]:
# may want to do some cleaning of the media table at a later time. Difficult to parse w/ g/mg/ug/ml
# units dispersed throughout the dataframe

media_table = media_table.fillna(0) # replace NA with 0
# conditions_only = media_table[media_conditions]

# for condition in media_conditions:
#     condition_series = conditions_only[condition]
#     condition_series.str.find('g')
# #conditions_only.find

# # convert grams to micrograms
# media_table = media_table.replace(to_replace=r'\s[g]', value=r'000000', regex=True)
# # convert mg to micrograms
# media_table = media_table.replace(to_replace=r'\s[mg]', value='000', regex=True)
# # strip microgram label
# media_table = media_table.replace(to_replace=r'\s[µg]', value='', regex=True)
# # convert masses from string to numeric

In [97]:
# which conditions had the most positive growth for these 10 species?
(growth_matrix.loc[study_organisms] != 0).sum()

GMM      10
BHI++    10
WCA      10
mGAM      9
M1        3
M2        8
M3        8
M4        7
M5        9
M7        7
M8       10
M9        0
M10       5
M11       9
M13       4
M14       4
M15A      4
M15B      4
M16       4
dtype: int64

In [91]:
# GMM, BHI++, WCA, and mGAM are the rich media used (the rest are defined).
# all other media are the only ones present in the media_table.

{'Amino acids',
 'Buffer compounds',
 'Nucleotids',
 'Others',
 'SCFA',
 'Salts & Minerals',
 'Sugar',
 'Vitamins & Antioxidants'}

In [92]:
media_table.columns

Index(['Class', 'Compound', 'M1: GMM', 'M2: LAB', 'M3: GMM+LAB',
       'M4: GMM+LAB low M/V', 'M5: GMM+LAB exclude SCFA',
       'M7: GMM+LAB only monosacharides', 'M8: GMM+LAB plus Mucin',
       'M9: GMM+LAB only Mucin', 'M10: GMM+LAB 10% aminoacids',
       'M11: GMM+LAB excluding aromatic AA', 'M13: B. thetaiotaomicron MM',
       'M14: C. perfiringens MM', 'M15A: E. coli MM1', 'M15B: E. coli MM2',
       'M16: V. parvula defined medium'],
      dtype='object')

In [94]:
growth_matrix.columns

Index(['GMM', 'BHI++', 'WCA', 'mGAM', 'M1', 'M2', 'M3', 'M4', 'M5', 'M7', 'M8',
       'M9', 'M10', 'M11', 'M13', 'M14', 'M15A', 'M15B', 'M16'],
      dtype='object')